In [1]:
import numpy as np
import pandas as pd
from molvs import standardize_smiles
from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit.Chem.Draw import MolsToGridImage, rdMolDraw2D, MolsMatrixToGridImage
from rdkit.Chem.Draw import IPythonConsole

In [2]:
df = pd.read_csv('data/compounds-parsed.tsv', delimiter='\t')
df

,entry,name,pathway,enzyme,dblinks,smiles
0,C00001,"['H2O', 'Water']","[('PATH', 'map00190', 'Oxidative phosphorylati...","['1.1.1.1', '1.1.1.22', '1.1.1.23', '1.1.1.115...","[('CAS', ['7732-18-5']), ('PubChem', ['3303'])...",O
1,C00002,"['ATP', ""Adenosine 5'-triphosphate""]","[('PATH', 'map00190', 'Oxidative phosphorylati...","['1.1.98.6', '1.2.1.30', '1.2.1.95', '1.2.1.10...","[('CAS', ['56-65-5']), ('PubChem', ['3304']), ...",Nc1ncnc2c1ncn2[C@@H]1O[C@H](COP(=O)(O)OP(=O)(O...
2,C00003,"['NAD+', 'NAD', 'Nicotinamide adenine dinucleo...","[('PATH', 'map00190', 'Oxidative phosphorylati...","['1.1.1.1', '1.1.1.3', '1.1.1.4', '1.1.1.6', '...","[('CAS', ['53-84-9']), ('PubChem', ['3305']), ...",NC(=O)c1ccc[n+]([C@@H]2O[C@H](COP(=O)(O)OP(=O)...
3,C00004,"['NADH', 'DPNH', 'Reduced nicotinamide adenine...","[('PATH', 'map00190', 'Oxidative phosphorylati...","['1.1.1.1', '1.1.1.3', '1.1.1.4', '1.1.1.6', '...","[('CAS', ['58-68-4']), ('PubChem', ['3306']), ...",NC(=O)C1=CN([C@@H]2O[C@H](COP(=O)(O)OP(=O)(O)O...
4,C00005,"['NADPH', 'TPNH', 'Reduced nicotinamide adenin...","[('PATH', 'map00195', 'Photosynthesis'), ('PAT...","['1.1.1.1', '1.1.1.2', '1.1.1.3', '1.1.1.10', ...","[('CAS', ['2646-71-1']), ('PubChem', ['3307'])...",NC(=O)C1=CN([C@@H]2O[C@H](COP(=O)(O)OP(=O)(O)O...
...,...,...,...,...,...,...
4765,C18354,['(S)-Chiral alcohol'],"[('PATH', 'map01057', 'Biosynthesis of type II...",['1.1.1.-'],"[('PubChem', ['124489028']), ('ChEBI', ['81700...",CC(=O)C1=C(C[C@H](O)CC(=O)O)Cc2cccc(O)c2C1=O
4766,C18356,['(S)-Hemiketal'],"[('PATH', 'map01057', 'Biosynthesis of type II...",['4.2.1.-'],"[('PubChem', ['124489030']), ('ChEBI', ['81702...",CC1(O)O[C@H](CC(=O)O)CC2=C1C(=O)c1c(O)cccc1C2
4767,C18358,"['DHKred-OH', 'Hydroxylated dihydrokalafungin ...","[('PATH', 'map01057', 'Biosynthesis of type II...",['1.14.14.-'],"[('PubChem', ['124489032']), ('ChEBI', ['81704...",C[C@H]1O[C@H](CC(=O)O)Cc2c1c(O)c1c(O)ccc(O)c1c2O
4768,C18359,['Dihydrogranaticin'],"[('PATH', 'map01057', 'Biosynthesis of type II...",NaN,"[('CAS', ['63999-06-4']), ('PubChem', ['124489...",C[C@@H]1O[C@@H](CC(=O)O)CC2=C1C(=O)c1c(O)c3c(c...


In [3]:
#Split salts into separate entries for each ion
df_split = df['smiles'].str.split('.', expand=False).explode().reset_index()
df = df.drop('smiles', axis=1).reset_index().merge(df_split, on='index')
df.drop('index', axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4789 entries, 0 to 4788
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   entry    4789 non-null   object
 1   name     4789 non-null   object
 2   pathway  4789 non-null   object
 3   enzyme   3962 non-null   object
 4   dblinks  4789 non-null   object
 5   smiles   4789 non-null   object
dtypes: object(6)
memory usage: 224.6+ KB


In [4]:
df['smiles'] = df['smiles'].apply(standardize_smiles)

[21:48:51] WARNING: not removing hydrogen atom without neighbors


In [5]:
df[['entry', 'smiles']].to_csv('data/to-ionize.tsv', sep='\t', index=False)

In [7]:
!python scripts/ionize.py


For help, use: python dimorphite_dl.py --help

If you use Dimorphite-DL in your research, please cite:
Ropp PJ, Kaminsky JC, Yablonski S, Durrant JD (2019) Dimorphite-DL: An
open-source program for enumerating the ionization states of drug-like small
molecules. J Cheminform 11:14. doi:10.1186/s13321-019-0336-9.


For help, use: python dimorphite_dl.py --help

If you use Dimorphite-DL in your research, please cite:
Ropp PJ, Kaminsky JC, Yablonski S, Durrant JD (2019) Dimorphite-DL: An
open-source program for enumerating the ionization states of drug-like small
molecules. J Cheminform 11:14. doi:10.1186/s13321-019-0336-9.


For help, use: python dimorphite_dl.py --help

If you use Dimorphite-DL in your research, please cite:
Ropp PJ, Kaminsky JC, Yablonski S, Durrant JD (2019) Dimorphite-DL: An
open-source program for enumerating the ionization states of drug-like small
molecules. J Cheminform 11:14. doi:10.1186/s13321-019-0336-9.


For help, use: python dimorphite_dl.py --help

If you 

In [4]:
df = pd.read_csv('data/ionized.tsv', delimiter='\t')

In [5]:
search_patterns = [
    '[CX3H1](=O)[#6]', #aldehyde
    '[OX2H][#6X3]=[#6]', #enole
    '[OX2,OX1-][OX2,OX1-]', #peroxyde
    '[O-][C,c;!$([C]=[!#6;!H])]' #not acidic hydroxyl ionized (phenoles and alcohols)
]

def filter_patterns(mol):
    for pattern in search_patterns:
        pattern = Chem.MolFromSmarts(pattern)
        if mol.HasSubstructMatch(pattern):
            return True
    return False

In [6]:
#Filter out banned patterns
df = df[~df['smiles'].apply(lambda x: filter_patterns(Chem.MolFromSmiles(x)))]
df

[16:42:22] WARNING: not removing hydrogen atom without neighbors


,kegg_id,smiles
0,C00001,O
1,C00002,Nc1ncnc2c1ncn2[C@@H]1O[C@H](COP(=O)(O)OP(=O)([...
8,C00002,Nc1ncnc2c1ncn2[C@@H]1O[C@H](COP(=O)(O)OP(=O)([...
10,C00002,Nc1ncnc2c1ncn2[C@@H]1O[C@H](COP(=O)(O)OP(=O)(O...
17,C00002,[NH3+]c1ncnc2c1ncn2[C@@H]1O[C@H](COP(=O)([O-])...
...,...,...
140782,C18358,C[C@H]1O[C@H](CC(=O)[O-])Cc2c1c(O)c1c(O)ccc(O)...
140821,C18359,C[C@@H]1O[C@@H](CC(=O)O)CC2=C1C(=O)c1c(O)c3c(c...
140823,C18359,C[C@@H]1O[C@@H](CC(=O)[O-])CC2=C1C(=O)c1c(O)c3...
140838,C18360,C[C@H]1O[C@H](CC(=O)O)Cc2c1c(O)c1c(O)cccc1c2O


In [7]:
df['formal_charge'] = df['smiles'].apply(lambda x: Chem.GetFormalCharge(Chem.MolFromSmiles(x)))
df = df[(df['formal_charge']>-3)&(df['formal_charge']<3)&(df['formal_charge']!=0)]

[16:42:47] WARNING: not removing hydrogen atom without neighbors


In [8]:
df.to_csv('data/ionized-filtered.tsv', sep='\t', index=False)

In [2]:
# allowed_elements = {
#     'C', 'c',
#     'H',
#     'O', 'o', 
#     'N', 'n', 
#     'S', 's', 
#     'F',
#     'P', 'p' 
#     'Cl', 'cl',
#     'Br', 'br', 
#     'I', 'i', 
#     'B', 'b',
# }

# exclude_patterns = [
# '[CX3H1](=O)', #Aldehyde 
# '[OX2,OX1-][OX2,OX1-]', #Peroxyde 
# '[OX2H][#6X3]=[#6]', #Enole 
# ]

In [ ]:
# search_patterns = [
# 'OS(=O)(=O)[OH]',
# '[!O]S(=O)(=O)[OH]',
# 'O[SD3](=O)[OH]',
# '[!O][SD3](=O)[OH]',
# 'OP(=O)([OH])[OH]',
# '[!O]P(=O)([OH])[OH]',
# 'C(=O)[OH]',
# 'OP(=O)([O-])[OH]',
# '[!O]P(=O)([O-])[OH]',
# 'OB([OH])[OH]',
# '[!O]B([OH])[OH]',
# '[c:1]-[NX3+0:2]',
# '[C:1]-[NX3+0:2]',
# '[N:1]-[C:2](-[N:3])=[NX2:4]',
# '[C:1](-[N:2])=[NX2+0:3]',
# '[SX3:1](=[O:2])-[O:3]',
# '[SX4:1](=[O:2])(=[O:3])([O:4]-[C,c,N,n:5])-[OX2:6]',
# '[SX4:1](=[O:2])(=[O:3])(-[C,c,N,n:4])-[OX2:5]'
# ]
# '[$(*-[NX2-]-[NX2+]#[NX1]),$(*-[NX2]=[NX2+]=[NX1-])]', #Azide 
# '[$([NX1-]=[NX2+]=[NX1-]),$([NX1]#[NX2+]-[NX1-2])]', #Azide ion 
# '[NX1]#[CX2]', #Nitlile      
# '[CX1-]#[NX2+]', #Isonitrile 
# '[N+][CX3](=[OX1])[#6]', #Protonated amide 
# '[$([#7+][OX1-]),$([#7v5]=[OX1]);!$([#7](~[O])~[O]);!$([#7]=[#7])]' #N oxyde
# '[$([#16+][OX1-]),$([#16v5]=[OX1]);!$([#16](~[O])~[O]);!$([#16]=[#16])]', #S oxyde
# 'c1cncnc1', #Nucleobases
# 'c1ncnc2ncnc12',
# 'NN',
# 'N=N'

In [175]:
# def find_pattern(mol):
#     pattern ='S(=O)(=O)(O)[O-]'
#     pattern = Chem.MolFromSmarts(pattern)
#     if mol.HasSubstructMatch(pattern):
#         return True
#     return False

In [22]:
# def filter_patterns(mol):
#     for pattern in search_patterns:
#         pattern = Chem.MolFromSmarts(pattern)
#         if mol.HasSubstructMatch(pattern):
#             return True
#     return False

In [16]:
# print(len(df[df['molecule'].apply(find_pattern)]['molecule'].tolist()))
# MolsToGridImage(mols = df[df['molecule'].apply(find_pattern)]['molecule'].tolist(),
#                molsPerRow=6)

In [82]:
# def check_charges(mol):
#     pos=0
#     neg=0
#     charge = Chem.GetFormalCharge(mol)
#     for atom in mol.GetAtoms():
#         if atom.GetFormalCharge() > 0:
#             pos+=1
#         elif atom.GetFormalCharge() < 0:
#             neg+=1
#     if charge > 0 and neg == 0:
#         return True
#     elif charge < 0 and pos == 0:
#         return True
#     # elif charge == 0 and neg!=0 and pos!=0:
#     #     return True
#     return False
# df = df[df['molecule'].apply(check_charges)]

In [7]:
# df['molWt'] = df['molecule'].apply(Chem.rdMolDescriptors.CalcExactMolWt)
# df['HBD'] = df['molecule'].apply(Chem.rdMolDescriptors.CalcNumHBD)
# df['HBA'] = df['molecule'].apply(Chem.rdMolDescriptors.CalcNumHBA)
# df.describe()

,molWt,HBD,HBA
count,5972.000000,5972.000000,5972.000000
mean,379.936823,3.763396,6.321333
std,299.081465,3.596801,6.090183
min,1.007276,0.000000,0.000000
25%,182.079038,1.000000,2.000000
50%,299.282429,3.000000,4.000000
75%,464.207183,5.000000,8.000000
max,3429.713307,52.000000,60.000000


In [71]:
# def filter_elements(mol):
#     if all(atom.GetSymbol() in allowed_elements for atom in mol.GetAtoms()):
#         return True
#     return False
    
# def filter_excluded_patterns(mol):
#     for pattern in exclude_patterns:
#         pattern = Chem.MolFromSmarts(pattern)
#         if mol.HasSubstructMatch(pattern):
#             return False
#     return True
    
# df = df[df['molecule'].apply(filter_elements)]
# df = df[df['molecule'].apply(filter_excluded_patterns)]
# df.info()